In [1]:
import pandas as pd
import openpyxl
from openpyxl import Workbook
import getpass

Importing Camper Dataset

In [2]:
def clean_dataframe(df):
    for time in ["9:00 AM", "10:00 AM","11:00 AM","1:00 PM", "2:00 PM","3:00 PM"]:
        lst = df[time]
        df[time]  = [x.rstrip() for x in lst]
    return df

In [3]:
def get_unique_classes_and_times(camper_df):
    hourly_class_dict = {"9:00 AM":[],"10:00 AM":[],"11:00 AM":[],"1:00 PM":[],"2:00 PM":[],"3:00 PM":[]}
    for key in hourly_class_dict:
        #getting classes in a given hour
        classes = camper_df[key].tolist()
        #dropping any whitespace
        classes = [x.rstrip() for x in classes]
        #getting unique values
        unique_classes = list(set(classes))
        #assigning unique classes to dictionary
        hourly_class_dict[key] = unique_classes
    return hourly_class_dict


In [4]:
#### column = A,B,C values = values to add
def add_column(sheet, column, values):
    values = list(values)
    for i in range(len(values)):
        value = column + str(i+2)
        sheet[value] = values[i]

In [5]:
def new_roster(participant_df,name,hour,course):
    workbook = openpyxl.load_workbook('correct template.xlsx')
    sheets = workbook.active
    #adding camper names and sign out option
    participant_df = participant_df.sort_values("Last Name")
    add_column(sheets,"A",participant_df["First Name"])
    add_column(sheets,"B",participant_df["Last Name"])
    add_column(sheets,"C",participant_df["Check Out Option"])
    #add activity name to check-in, out, and attendence 
    check_in = workbook["Check In"]
    check_in["A9"] = course

    check_out = workbook["Checkout"]
    check_out["A9"] = course
    
    attendence = workbook["Attendence Roster "]
    attendence["B1"] = hour+ " Activity Roster"
    attendence["B5"] = course


    save_path = '/Users/'+getpass.getuser()+'/Documents/Blue Camp/Output/'+name + '.xlsx'
    workbook.save(filename=save_path)


In [6]:
def roster_iterator(classes_dict,participants):
    for key in classes_dict:
        for course in classes_dict[key]:
            roster_name = key + " " + course
            class_participants_df = participants[participants[key]==course]
            new_roster(class_participants_df,roster_name,key,course)

Pulling it all together

In [7]:
def create_rosters(camper_path):
    #reading in camper data
    campers_df = pd.read_csv(camper_path)
    campers_df = clean_dataframe(campers_df)
    #getting unique classes
    unique_classes_dict = get_unique_classes_and_times(campers_df)
    #iteratively generating rosters
    roster_iterator(unique_classes_dict,campers_df)

Calling Function

In [8]:
create_rosters('/Users/cameronfaulkner/Downloads/B&G Session 4 (Weeks 7&8) 2023 - B_G W8 (One Weekers).csv')